In [0]:
#!pip install pytz
import pandas as pd
import pytz
import datetime as dt
import requests

BASE_URL = "http://api.openweathermap.org/data/2.5/weather?"
API_KEY = "API KEY"
CITY = "bangalore"

def kelvin_to_celsius_fahernheit(kelvin):
  celsius = kelvin - 273.15
  fahernheit = celsius * (9/5) + 32
  return celsius, fahernheit

url = BASE_URL + "appid=" + API_KEY + "&q=" + CITY
response = requests.get(url).json()

temp_kelvin = response["main"]["temp"]
temp_celsius, temp_fahrenheit = kelvin_to_celsius_fahernheit(temp_kelvin)
feels_like_kelvin = response["main"]["feels_like"]
feels_like_celsius, feels_like_fahrenheit = kelvin_to_celsius_fahernheit(feels_like_kelvin)
wind_speed = response["wind"]["speed"]
humidity = response["main"]["humidity"]
description = response["weather"][0]["description"]
sunrise_time = dt.datetime.utcfromtimestamp(response["sys"]["sunrise"] + response["timezone"])
sunset_time = dt.datetime.utcfromtimestamp(response["sys"]["sunset"] + response["timezone"])
current_time= pytz.timezone('Asia/Kolkata')
datetime_Mumbai = dt.datetime.now(current_time).strftime('%Y-%m-%d %H:%M:%S').split('.')[0]
import pandas as pd

# Import pandas
import pandas as pd

# Create DataFrame
weather_data = {
    'Information': [
        f"Temperature in {CITY}: {temp_celsius:.2f}°C or {temp_fahrenheit:.2f}°F",
        f"Temperature in {CITY} feels like: {feels_like_celsius:.2f}°C or {feels_like_fahrenheit:.2f}°F",
        f"Humidity in {CITY}: {humidity}%",
        f"Wind Speed in {CITY}: {wind_speed} m/s",
        f"General Weather in {CITY}: {description}",
        f"Current date and time in {CITY}: {datetime_Mumbai}",
        f"Sunrises in {CITY}: {sunrise_time} local time.",
        f"Sunsets in {CITY}: {sunset_time} local time."
    ]
}

df = pd.DataFrame(weather_data)

# Set display options for pandas
pd.set_option('display.max_colwidth', None)  # Show full contents of each cell

# Print DataFrame
df
df.to_csv("weather.csv",index = False)

In [0]:
dbutils.fs.ls('file:/databricks/driver')

Out[11]: [FileInfo(path='file:/databricks/driver/azure/', name='azure/', size=4096, modificationTime=1710841791568),
 FileInfo(path='file:/databricks/driver/hadoop_accessed_config.lst', name='hadoop_accessed_config.lst', size=2755, modificationTime=1710841791584),
 FileInfo(path='file:/databricks/driver/preload_class.lst', name='preload_class.lst', size=1306936, modificationTime=1710841791580),
 FileInfo(path='file:/databricks/driver/conf/', name='conf/', size=4096, modificationTime=1710841789504),
 FileInfo(path='file:/databricks/driver/eventlogs/', name='eventlogs/', size=4096, modificationTime=1710842044990),
 FileInfo(path='file:/databricks/driver/logs/', name='logs/', size=4096, modificationTime=1710842509621),
 FileInfo(path='file:/databricks/driver/weather.csv', name='weather.csv', size=383, modificationTime=1710843729623),
 FileInfo(path='file:/databricks/driver/ganglia/', name='ganglia/', size=4096, modificationTime=1710843303176),
 FileInfo(path='file:/databricks/driver/metas

In [0]:
dbutils.fs.mv('file:/databricks/driver/weather.csv', '/weather.csv', recurse=True)

Out[12]: True

In [0]:
%pip install snowflake
%pip install snowflake-connector-python
%pip install snowflake-connector-python
import pandas as pd
from pyspark.sql import SparkSession
import snowflake.connector

# Snowflake connection parameters
account = 'My Account'
user = 'My User Name'
password = 'My Password'
warehouse = 'My Warehouse Name'
database = 'My Database Name'
schema = 'My Schema Name'
table_name = 'My Table Name'  # Table name without '.csv' suffix

# Snowflake URL
snowflake_url = ''

# Load CSV into a Spark DataFrame
file_path = "/weather.csv"
spark = SparkSession.builder.appName("SnowflakeIntegration").getOrCreate()
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Snowflake connection
conn = snowflake.connector.connect(
    user=user,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema,
    sfURL=snowflake_url  # Include Snowflake URL
)

# Create cursor
cur = conn.cursor()

# Replace spaces and dots in table name
table_name_sanitized = table_name.replace(' ', '_').replace('.', '_')

# Create table in Snowflake with dynamically generated column definitions
column_definitions = ', '.join([f'"{col}" STRING' for col in df.columns])
cur.execute(f'CREATE OR REPLACE TABLE "{schema}"."{table_name_sanitized}" ({column_definitions})')

# Write DataFrame to Snowflake table
df.write.format("snowflake") \
    .option("sfURL", snowflake_url) \
    .option("sfUser", user) \
    .option("sfPassword", password) \
    .option("sfDatabase", database) \
    .option("sfSchema", schema) \
    .option("sfWarehouse", warehouse) \
    .option("dbtable", table_name_sanitized) \
    .mode("overwrite") \
    .save()

# Commit the transaction
conn.commit()

# Close cursor and connection
cur.close()
conn.close()


Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
